In [ ]:
#required packages
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from pymc3 import Model
import pymc3 as pm
import theano
import theano.tensor as tt
import datetime

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
# Load dataset
# df = pd.read_csv('denver_prop_with_ct_clean_small.csv')
df = pd.read_csv('../data/denver_dataset_milestone3.csv')

#convert to datetime format
df["list_date"] = pd.to_datetime(df["list_date"])
df["sale_date"] = pd.to_datetime(df["sale_date"])

df = df[-np.isnan(df['longitude'])]

df

,Unnamed: 0,18-59,mean_household_income,built 1995 or later,OTHER,mobile_home_pct,annual_births_per_resident,farm_score,luxury_communities_score,CONDO,...,bathfull,small_apt_buildings_pct,standardized_test_score_percentile,MULTI_FAMILY,bedrooms,list_date,sale_date,rex_property_id,latitude,longitude
0,0,0.616287,107073,80.577849,False,0.000000,0.016527,100,21.852379,False,...,3,0.802568,81.596980,False,4,2016-06-30,2016-08-12,207973152,39.61454,-104.72075
1,1,0.616287,107073,80.577849,False,0.000000,0.016527,100,21.852379,False,...,3,0.802568,81.596980,False,4,2019-04-26,2019-05-29,207973152,39.61454,-104.72075
2,4,0.616287,107073,80.577849,False,0.000000,0.016527,100,21.852379,False,...,2,0.802568,81.596980,False,5,2020-06-26,2020-08-10,208397574,39.61368,-104.72888
3,5,0.616287,107073,80.577849,False,0.000000,0.016527,100,21.852379,False,...,3,0.802568,81.596980,False,3,2017-04-21,2017-05-15,207998741,39.62276,-104.72723
4,6,0.616287,107073,80.577849,False,0.000000,0.016527,100,21.852379,False,...,4,0.802568,81.596980,False,4,2017-06-16,2017-07-14,207964420,39.62257,-104.72502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218364,277545,0.623831,108207,3.015682,False,0.422195,0.016525,100,16.930720,True,...,2,0.000000,81.854093,False,2,2018-05-18,2018-06-28,352759060,39.90318,-105.05226
218365,277548,0.541232,78117,22.450889,False,8.840037,0.018905,0,50.545896,False,...,1,5.051450,42.274687,False,3,2020-09-08,2100-01-01,121948410,40.58694,-104.74366
218366,277549,0.519823,118127,40.666667,False,9.638889,0.008756,0,26.129220,False,...,4,0.527778,74.072710,False,4,2017-05-31,2100-01-01,122775824,40.19702,-105.02302
218367,277550,0.511367,112043,27.410359,False,2.788845,0.009153,100,25.255845,False,...,1,0.876494,60.597145,False,4,2017-06-02,2100-01-01,93936156,39.33200,-104.66001


Current approach is trained on a particular discretized timeframe (say a quarter / 3 months). One potential exploration is to train the heirarchical models on separate discretized quarters and generate a time series on the changing sub-market classifications across the quarters. However, one concern is when a house is sold before a particular quarter and the model would assume that its corresponding sub-market has low supply?

In [ ]:
def gen_y(t_disc, data, t0=None):
    ''' 
    t_disc: datetime.timedelta(days = XX)
    t0: datetime.datetime(YYYY,MM,DD)
    '''

    if t0 is not None:
        listed = np.array(((data['list_date'] >= t0) & (data['list_date'] < t0 + t_disc)) | ((data['list_date'] < t0) & (data['sale_date'] >= t0)), dtype=np.int8)
        sale = np.array((data['sale_date'] >= t0) & (data['sale_date'] < t0 + t_disc), dtype = np.int8)
        return np.vstack((listed, sale)).T
    #else:
        #TODO


In [ ]:
y_2019Q2 = gen_y(datetime.timedelta(days = 90), df, datetime.datetime(2019,4,1))

# Remove all rows that aren't listed in that period
listed_index = np.where(y_2019Q2[:,0] == 1)
df_2019Q2 = df.iloc[listed_index]

df_2019Q2 = df_2019Q2.drop(columns=['rex_property_id','Unnamed: 0'])


# Build X matrix and Y target vector
X = df_2019Q2.drop(columns=['list_date','sale_date','latitude','longitude'])
X = X.reset_index(drop=True)

Y = y_2019Q2[np.where(y_2019Q2[:,0]==1)][:,0]

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34107 entries, 0 to 34106
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   18-59                               34107 non-null  float64
 1   mean_household_income               34107 non-null  int64  
 2   built 1995 or later                 34107 non-null  float64
 3   OTHER                               34107 non-null  bool   
 4   mobile_home_pct                     34107 non-null  float64
 5   annual_births_per_resident          34107 non-null  float64
 6   farm_score                          34107 non-null  int64  
 7   luxury_communities_score            34107 non-null  float64
 8   CONDO                               34107 non-null  bool   
 9   property_crime_rate                 34107 non-null  float64
 10  bathfull                            34107 non-null  int64  
 11  small_apt_buildings_pct             34107

In [ ]:
X = X.to_numpy(dtype=np.float64)
X.shape

(34107, 15)

In [ ]:
Y.shape

(34107,)

In [ ]:
K = 5 # Number of submarkets
n_homes, n_features = X.shape
mu_k_prior_var = 10
sigma_k_prior_var = 10
mu_b1_prior_var = 1
sigma_b1_prior_var = 1

with Model() as baseline_mod:

    # Submarket latent variable
    p = pm.Dirichlet("p", a=np.ones(K)/K, shape=K)

    submarket = pm.Categorical('submarket', p=p, shape=n_homes)

    # Generate homes features 
    mu_k = pm.Normal('mu_k', mu=0, sigma=mu_k_prior_var, shape=(K,n_features))

    sigma_k = pm.Normal('sigma_k', mu=0, sigma=sigma_k_prior_var, shape=(n_features,n_features))

    homes = pm.MvNormal('homes', mu_k[submarket,:], sigma_k, observed=X)

    # Generate betas for "hedonic" regression
    mu_b1 = pm.Normal('mu_b1', mu=0, sigma=mu_b1_prior_var, shape=(K,n_features))

    sigma_b1 = pm.Normal('sigma_b1', mu=0, sigma=sigma_b1_prior_var, shape=(n_features,n_features))

    beta1 = pm.MvNormal('beta1', mu_b1[submarket,:], sigma_b1, shape=(n_homes,n_features))

    # Final layer
    y1 = pm.Bernoulli('y1', p=1/(1+tt.exp(-tt.tensordot(homes,tt.transpose(beta1)))), observed = Y)

In [ ]:
init_clustering = KMeans(n_clusters=K, random_state=0).fit(X)
value, count = np.unique(init_clustering.labels_, return_counts = True)
print(value)
print(count)

[0 1 2 3 4]
[12622  8921 10888   346  1330]


In [ ]:
# theano.config.exception_verbosity='high'

with baseline_mod:

    step1 = pm.Metropolis(vars=[p, mu_k, sigma_k, homes, mu_b1, sigma_b1, beta1, y1])
    step2 = pm.ElemwiseCategorical(vars=[submarket], values=np.arange(K))
    tr = pm.sample(1000, tune=1000, cores=2, step=[step1, step2], start={"submarket": init_clustering.labels_, "sigma_k": np.eye(n_features), "sigma_b1": np.eye(n_features)}, compute_convergence_checks=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: ElemwiseCategorical is deprecated, switch to CategoricalGibbsMetropolis.
  
/root/venv/lib/python3.7/site-packages/pymc3/sampling.py:469: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>CompoundStep
>>Metropolis: [beta1]
>>Metropolis: [sigma_b1]
>>Metropolis: [mu_b1]
>>Metropolis: [sigma_k]
>>Metropolis: [mu_k]
>>Metropolis: [p]
>ElemwiseCategorical: [submarket]


/root/venv/lib/python3.7/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),
/root/venv/lib/python3.7/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),


ValueError: Not enough samples to build a trace.

In [ ]:
len(tr['submarket'])

2000

In [ ]:
values = []
counts = []

for i in range(20):
    value, count = np.unique(tr['submarket'][i+500], return_counts = True)
    values.append(value)
    counts.append(count)

In [ ]:
values

[array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4]),
 array([4])]

In [ ]:
counts

[array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107]),
 array([34107])]

In [ ]:
denver = plt.imread('denver_map.png')
X_sub1 = df_2019Q2[tr['submarket'][1500] == 0]
X_sub2 = df_2019Q2[tr['submarket'][1500] == 1]
X_sub3 = df_2019Q2[tr['submarket'][1500] == 2]
BBox = (X_sub1.longitude.min(), X_sub1.longitude.max(),      
         X_sub1.latitude.min(), X_sub1.latitude.max())
        
fig, ax = plt.subplots(figsize = (10,10))

my_cmap = plt.cm.YlOrBr
my_cmap.set_under('w',1)

ax.scatter(X_sub1.longitude, X_sub1.latitude, color='r', alpha= 0.3, cmap=my_cmap, label="Submarket 1")
ax.scatter(X_sub2.longitude, X_sub2.latitude, color='b', alpha= 0.3, cmap=my_cmap, label="Submarket 2")
ax.scatter(X_sub3.longitude, X_sub3.latitude, color='g', alpha= 0.5, cmap=my_cmap, label="Submarket 3")
ax.set_title('Plotting Spatial Data on Denver Map')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.legend(loc='best')
ax.imshow(denver, zorder=0, extent = BBox, aspect= 'equal')

plt.show()

In [ ]:
def submarket_pymc3(trace_category):
    
    values = np.unique(trace_category)

    for i in range(len(values)):

        X_sub = X[trace_category == values[i]]
        Y_sub = Y[trace_category == values[i]]
        
        # Ensure that sub-market has both classifications
        labels = np.unique(Y_sub)
        if len(labels) == 1:
            continue
            
        X_train, X_test, y_train, y_test = train_test_split(X_sub, Y_sub, test_size=0.3)

        model_sub = LogisticRegression().fit(X_train, y_train[:,1])

        train_acc = model_sub.score(X_train, y_train[:,1])
        test_acc = model_sub.score(X_test, y_test[:,1])
        auc = roc_auc_score(y_test[:,1], model_sub.predict(X_test)) 
        
        exp_sales = sum(model_sub.predict_proba(X_test)[:,1])
        actual_sales = sum(y_test[:,1])

        print("Sub-Market #{} Demand Prediction".format(i+1))
        print("Number of Homes: {}".format(len(Y_sub)))
        print("Training Accuracy: {:.3f}%".format(train_acc*100))
        print("--- Testing ---")
        print("Testing Accuracy: {:.3f}%".format(test_acc*100))
        print("AUC: {:.3f}".format(auc))
        print("Expected #Sales: {}".format(round(exp_sales)))
        print("Actual #Sales: {}\n".format(actual_sales))

In [ ]:
submarket_pymc3(tr['submarket'][500])

In [ ]:
np.unique(tr['submarket'][-1], return_counts = True)

(array([3]), array([34107]))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1c850c61-d934-4c85-b16d-3cb283df0c84' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>